# Yfinance, Option Chain, and Data

Focus: Gain familiarity with Yfinance, the option chain data, and how to manipulate the data for later computations of implied volatilty surfaces and PCA.

Notes:
- The ability to calculate an implied volatity through BS, simple samples, and rootfinding methods has already been implemented at this point 

- The focus of this notebook will be to further these implementations with real market data

- There will be conceptual note for both myself and other to follow along in this notebook

In [4]:
#Allow imports from the src directory
import sys
from pathlib import Path
project_root = Path().resolve().parents[0]
sys.path.append(str(project_root))

# Import self-made libraries to run checks
from src.black_scholes import black_scholes_price
from src.implied_vol import implied_volatility

# Standard imports
import yfinance as yf
import pandas as pd
import numpy as np

Cell 1 Focus/Notes:  
<br>
.option_chain()
- Have to pass in a date of string type labeled as "YYYY-MM-DD"
- Cannot pull multiple dates without a loop and adding to data frame

Option chain data
- Pulls IV but also prints in decimal form not percent
- Ex. Yfinance IV = 2.98 = 298%  
<br>
- Bid is the highest premium someone in the market is willing to pay for the option contract
- Ask is the lowest premium someon will accept to sell the contract
- Mid price = (bid+ask)/2 which is our fair market price estimate  
<br>
- Volume: How many contracts traded today
- Open interest: How many contracts still open and active
- Contract size: Shares per contract (REGULAR =  100 shares)



In [5]:
#Initialization 
ticker = "SPY"
tk = yf.Ticker(ticker)

#Example of not using a specific expiration date
SPYdf = tk.option_chain()
print("SPYdf data type:", type(SPYdf), "<- Notes that it does not return a DataFrame. Instead: <class 'tuple'>\n")

#Pulling available expiration dates
expirations = tk.options
print("SPY Expirations:", expirations, "\n")

#Pull option chain for a specific date
SPYdf = tk.option_chain('2026-02-20')
print("SPYdf data type:", type(SPYdf), "<- Now its a pndas DataFrame\n")
print("View of dataframe:\n")
print(SPYdf)

SPYdf data type: <class 'yfinance.ticker.Options'> <- Notes that it does not return a DataFrame. Instead: <class 'tuple'>

SPY Expirations: ('2026-02-12', '2026-02-13', '2026-02-17', '2026-02-18', '2026-02-19', '2026-02-20', '2026-02-23', '2026-02-24', '2026-02-25', '2026-02-27', '2026-03-06', '2026-03-13', '2026-03-20', '2026-03-27', '2026-03-31', '2026-04-17', '2026-04-30', '2026-05-15', '2026-05-29', '2026-06-18', '2026-06-30', '2026-07-31', '2026-08-21', '2026-09-18', '2026-09-30', '2026-12-18', '2026-12-31', '2027-01-15', '2027-03-19', '2027-06-17', '2027-12-17', '2028-01-21', '2028-06-16', '2028-12-15') 

SPYdf data type: <class 'yfinance.ticker.Options'> <- Now its a pndas DataFrame

View of dataframe:

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0    SPY260220C00335000 2026-02-02 19:44:19+00:00   335.0     362.17  345.41   
1    SPY260220C00345000 2026-02-02 14:42:12+00:00   345.0     348.01  335.42   
2    SPY260220C00350000 20

In [6]:
#Separating calls and puts into their own DataFrames & do small view
calls = SPYdf.calls
puts = SPYdf.puts
print("\nCalls DataFrame:\n", calls.head(5))
print("\nPuts DataFrame:\n", puts.head(5))

# View of columns in calls DataFrames
print("\nCalls DataFrame columns:\n", calls.columns)
print("There are ", len(calls), " call option for SPY with expiration date 2026-02-20\n")
print("There are ", len(puts), " put option for SPY with expiration date 2026-02-20\n")


Calls DataFrame:
        contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0  SPY260220C00335000 2026-02-02 19:44:19+00:00   335.0     362.17  345.41   
1  SPY260220C00345000 2026-02-02 14:42:12+00:00   345.0     348.01  335.42   
2  SPY260220C00350000 2026-02-03 19:25:15+00:00   350.0     337.41  330.43   
3  SPY260220C00365000 2025-11-26 15:41:15+00:00   365.0     316.48  325.28   
4  SPY260220C00370000 2025-11-20 17:07:06+00:00   370.0     292.80  320.42   

      ask  change  percentChange  volume  openInterest  impliedVolatility  \
0  348.49     0.0            0.0     1.0             3           2.028814   
1  338.53     0.0            0.0     2.0             2           1.961426   
2  333.53     0.0            0.0     7.0            12           1.925782   
3  328.67     0.0            0.0     2.0             2           3.058596   
4  323.70     0.0            0.0     NaN             1           3.010012   

   inTheMoney contractSize currency  
0        Tr

# Implied Volatility Check
Here I am going to check that my implied volatity caluclations matches the values given by the Yfinance option chain

In [8]:
#Imports
from src.risk_free_rate import calculate_risk_free_rate
from datetime import datetime, timezone

#Get current stock price
tk = yf.Ticker("SPY")
S = tk.info.get("regularMarketPrice", None)
if S is None:
    S = tk.history(period="1d")["Close"].iloc[-1]

#Calculate risk-free rate
r = calculate_risk_free_rate()

#Get current time in UTC
now = datetime.now(timezone.utc)
print(now)
print("now data type:", type(now))

#Calculate time to expiration
exp = "2026-02-20"
exp_dt = datetime.strptime(exp, "%Y-%m-%d").replace(tzinfo=timezone.utc)
print("exp_dt data type:", type(exp_dt), "\n")

T = ((exp_dt - now).total_seconds()) / ((365.0 * 24 * 60 * 60))  # Time to expiration in years
print("Time to expiration T (in years):", T, "\n")
print("T data type:", type(T), "\n")


first100_calls = calls.head(100)
first100_puts = puts.head(100)

#Compare Yfinance IV with my calculated IV for calls - using mid price as the option price for IV calculation
print('\n\nComparing IV for 100 calls - using mid price \n\n')
for row in first100_calls.itertuples(index=False):
    bid = row.bid
    ask = row.ask
    K = row.strike
    yahoo_iv = row.impliedVolatility  # already decimal 
    mid = 0.5 * (bid + ask)

    # ---- My implied vol function call ----
    # Adjust this line to match your function signature
    calc_iv = implied_volatility(price=mid, S=S, K=K, T=T, r=r, option_type="call")

    print(
        f"K={K:>7.2f} mid={mid:>8.3f} | YahooIV={yahoo_iv:>8.4f}  CalcIV={calc_iv:>8.4f}  Diff={calc_iv - yahoo_iv:+.4f}"
    )


#Now Put options - using mid price as the option price for IV calculation
print('\n\nComparing IV for 100 puts - using mid price \n\n')
for row in first100_puts.itertuples(index=False):
    bid = row.bid
    ask = row.ask
    K = row.strike
    yahoo_iv = row.impliedVolatility  # already decimal 
    mid = 0.5 * (bid + ask)

    # ---- My implied vol function call ----
    # Adjust this line to match your function signature
    calc_iv = implied_volatility(price=mid, S=S, K=K, T=T, r=r, option_type="call")

    print(
        f"K={K:>7.2f} mid={mid:>8.3f} | YahooIV={yahoo_iv:>8.4f}  CalcIV={calc_iv:>8.4f}  Diff={calc_iv - yahoo_iv:+.4f}"
    )


#Compare Yfinance IV with my calculated IV for calls - using last price as the option price for IV calculation
print('\n\nComparing IV for 100 calls - using last price \n\n')
for row in first100_calls.itertuples(index=False):
    bid = row.bid
    ask = row.ask
    K = row.strike
    yahoo_iv = row.impliedVolatility  # already decimal 
    price = row.lastPrice

    # ---- My implied vol function call ----
    # Adjust this line to match your function signature
    calc_iv = implied_volatility(price=price, S=S, K=K, T=T, r=r, option_type="call")

    print(
        f"K={K:>7.2f} price={price:>8.3f} | YahooIV={yahoo_iv:>8.4f}  CalcIV={calc_iv:>8.4f}  Diff={calc_iv - yahoo_iv:+.4f}"
    )

#now with puts - using last price as the option price for IV calculation
print('\n\nComparing IV for 100 puts - using last price \n\n')
for row in first100_puts.itertuples(index=False):
    bid = row.bid
    ask = row.ask
    K = row.strike
    yahoo_iv = row.impliedVolatility  # already decimal 
    price = row.lastPrice

    # ---- My implied vol function call ----
    # Adjust this line to match your function signature
    calc_iv = implied_volatility(price=price, S=S, K=K, T=T, r=r, option_type="call")

    print(
        f"K={K:>7.2f} price={price:>8.3f} | YahooIV={yahoo_iv:>8.4f}  CalcIV={calc_iv:>8.4f}  Diff={calc_iv - yahoo_iv:+.4f}"
    )

2026-02-12 23:17:33.824098+00:00
now data type: <class 'datetime.datetime'>
exp_dt data type: <class 'datetime.datetime'> 

Time to expiration T (in years): 0.019258820899987317 

T data type: <class 'float'> 



Comparing IV for 100 calls - using mid price 


K= 335.00 mid= 346.950 | YahooIV=  2.0288  CalcIV=  2.1763  Diff=+0.1474
K= 345.00 mid= 336.975 | YahooIV=  1.9614  CalcIV=  2.1037  Diff=+0.1423
K= 350.00 mid= 331.980 | YahooIV=  1.9258  CalcIV=  2.0637  Diff=+0.1379
K= 365.00 mid= 326.975 | YahooIV=  3.0586  CalcIV=  3.4421  Diff=+0.3835
K= 370.00 mid= 322.060 | YahooIV=  3.0100  CalcIV=  3.3871  Diff=+0.3771
K= 375.00 mid= 307.005 | YahooIV=  1.7520  CalcIV=  1.8715  Diff=+0.1196
K= 380.00 mid= 302.000 | YahooIV=  1.7153  CalcIV=  1.8294  Diff=+0.1141
K= 385.00 mid= 301.790 | YahooIV=  2.4114  CalcIV=  2.7013  Diff=+0.2899
K= 390.00 mid= 296.825 | YahooIV=  2.3689  CalcIV=  2.6532  Diff=+0.2843
K= 395.00 mid= 287.030 | YahooIV=  1.6221  CalcIV=  1.7283  Diff=+0.1062
K= 400.00